In [ ]:
import os
import pandas as pd
import pydicom 
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

def display_all_images_and_explanations_with_boxes(csv_file, explanation_folder, dicom_folder):
    """
    Display all explanation images and their corresponding DICOM images side by side.
    :param csv_file: Path to the CSV file containing bounding box data.
    :param explanation_folder: Path to the folder containing explanation images.
    :param dicom_folder: Path to the folder containing DICOM files.
    """
    
    df = pd.read_csv(csv_file)
    
    original_width, original_height = 1024, 1024
    explanation_width, explanation_height = 369, 369
    
    # Scaling factors for explanation image
    scale_x = explanation_width / original_width
    scale_y = explanation_height / original_height

    # Iterate over all PNG files in the explanation folder
    for filename in os.listdir(explanation_folder):
        if filename.endswith(".png"):
            patient_id = filename.replace("_explanation.png", "")
            
            explanation_path = os.path.join(explanation_folder, filename)
            dicom_path = os.path.join(dicom_folder, f"{patient_id}.dcm")
            
            if not os.path.exists(dicom_path):
                print(f"DICOM file not found for patient ID: {patient_id}")
                continue

            # Load the explanation image
            explanation_image = Image.open(explanation_path)

            # Load the DICOM image
            dicom = pydicom.dcmread(dicom_path)
            dicom_image = dicom.pixel_array

            # Plot side by side
            plt.figure(figsize=(12, 6))

            # DICOM image
            plt.subplot(1, 2, 1)
            plt.imshow(dicom_image, cmap="gray")
            plt.title(f"DICOM Image - {patient_id}")
            plt.axis("off")
            
            patient_data = df[df["patientId"] == patient_id]
            for _, row in patient_data.iterrows():
                x, y, width, height = row["x"], row["y"], row["width"], row["height"]
                rect = plt.Rectangle((x, y), width, height, linewidth=2, edgecolor='red', facecolor='none')
                plt.gca().add_patch(rect)


            # Explanation image
            plt.subplot(1, 2, 2)
            plt.imshow(explanation_image)
            plt.title(f"Explanation Image - {patient_id}")
            plt.axis("off")
            
            explanation_draw = ImageDraw.Draw(explanation_image)
            for _, row in patient_data.iterrows():
                x, y, width, height = row["x"] * scale_x, row["y"] * scale_y, row["width"] * scale_x, row["height"] * scale_y
                explanation_draw.rectangle([(x, y), (x + width, y + height)], outline="red", width=2)

            plt.imshow(explanation_image)

            # Show plot
            plt.tight_layout()
            plt.show()

# Example usage
csv_file = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
dicom_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
explanation_folder = r"C:\Users\Admin\Documents\MasterThesis\prediction_images_2D_Fold2\onePrediction"
explanation_folder_false = r"C:\Users\Admin\Documents\MasterThesis\prediction_images_2D_Fold2\zeroPredictionFalse" # they are predicted 0 but are labeled 1
explanation_folder_no_pneumonia = r"C:\Users\Admin\Documents\MasterThesis\prediction_images_2D_Fold2\zeroPrediction"
explanation_folder_no_pneumonia_false = r"C:\Users\Admin\Documents\MasterThesis\prediction_images_2D_Fold2\onePredictionFalse"


# Displays images that have bounding boxes and are predicted correctly
#display_all_images_and_explanations_with_boxes(csv_file, explanation_folder, dicom_folder)
display_all_images_and_explanations_with_boxes(csv_file, explanation_folder_false, dicom_folder)
#display_all_images_and_explanations_with_boxes(csv_file, explanation_folder_no_pneumonia, dicom_folder)
#display_all_images_and_explanations_with_boxes(csv_file, explanation_folder_no_pneumonia_false, dicom_folder)